In [86]:
from algothon2021long_short import LongShort
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

assets = {}
regressions = {}
for i in range(1,6):
    assets[i] = pd.read_csv(f'long_short_training_features_targets/sector 1 - tech/Asset_{i}_training_data.csv')
    y = assets[i]['target']
    X = assets[i].drop(columns = ['target'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    regressions[i] = LinearRegression().fit(X_train, y_train)
    print('mse', sum((regressions[i].predict(X_test) - y_test)**2)/len(y_test))
    plt.figure()
    plt.plot(regressions[i].predict(X_test),y_test)

In [111]:
class myLongShort(LongShort):
    def __init__(self, **params):
        super(myLongShort, self).__init__(**params)

        # these variables can help you identify the asset and day that you are predicting for

        self.asset = 1 # asset number, going from 1 to 5
        self.day = 1 # day at which you are predicting of the series
        self.validate_max_days = 377 # number of days available per asset in the validation set
        self.test_max_days = 376 # number of days available per asset in the test set

        self.predictions = np.zeros((self.validate_max_days, 5)) # array that mirros the predictions of the API
        self.dataset = pd.DataFrame() 

    def train(self):
        self.assets = {}
        self.regressions = {}
        for i in range(1,6):
            self.assets[i] = pd.read_csv(f'long_short_training_features_targets/sector 1 - tech/Asset_{i}_training_data.csv')
            y = self.assets[i]['target']
            X = self.assets[i].drop(columns = ['target'])
            #f = 0.05
            #X_train = X[0:int(len(X)*f)]
            #X_test = X[int(len(X)*f):]
            #y_train = y[0:int(len(X)*f)]
            #y_test = y[int(len(X)*f):]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            
            self.regressions[i] = LinearRegression().fit(X_train, y_train)
            print('mse', sum((self.regressions[i].predict(X_test) - y_test)**2)/len(y_test))
            
    
    def predict2(self, data):
        data = pd.DataFrame(data).T
        data['Asset']=self.asset
        if len(self.dataset)==0:
            self.dataset = data
        else:
            self.dataset = pd.concat([self.dataset, data], axis=0)
        return 0
        
    
    def predict(self, data):
        data = pd.DataFrame(data).T
        # the run_validate and run_test functions take one asset and predict their weightings 
        # for all time indices before moving on to the next asset. This can make it harder to keep
        # the neutrality constraint. 

        # these if statements are iterating through the dataset, so that you know the asset and time index
        # that you are predicting for.

        if self.day < self.validate_max_days: # set this to self.test_max_days if using run_test()
            #print("Predicting for Asset {}".format(self.asset), "at Day {}".format(self.day))
            self.day += 1
            self.predictions[self.day-1, self.asset-2] = self.regressions[self.asset].predict(data)[0]# np.random.random() # you can compute your weights here

        elif self.day == self.validate_max_days and self.asset < 5:
            self.asset += 1
            #print("Predicting for Asset {}".format(self.asset), "at Day {}".format(self.day))
            self.day = 1
            self.predictions[self.day-1, self.asset-2] = self.regressions[self.asset].predict(data)[0] # you can compute your weights also here

        else:
            for i in range(self.predictions.shape[0]):
                
                argmin = self.predictions[i, :].argmin()
                argmax = self.predictions[i, :].argmax()
                self.predictions[i, :] *= 0
                self.predictions[i, argmin] = -1
                self.predictions[i, argmax] = 1
                
                #self.predictions[i, :] -= np.mean(self.predictions[i, :]) # subtract the mean to impose neutrality
                #self.predictions[i, :] /= np.abs(self.predictions[i, :]).max()
                #print(np.sum(self.predictions[i, :])) # check for neutrality

            pred_df = pd.DataFrame(self.predictions, columns = ["Asset {}".format(j) for j in range(1, 6)])
            pred_df.to_csv("qcap_validation_long_short_arg.csv", index=False)

        return 0

In [112]:
model = myLongShort(sector=1, validation_predictions_file="validation_predictions.csv",test_predictions_file = "test_predictions.csv")

In [113]:
model.train()

mse 6.36708428619813e-32
mse 1.0721148689360991e-32
mse 1.320544012055314e-32
mse 1.6029804864124593e-32
mse 6.046380350099021e-32


In [115]:
model.run_validate()

,1,2,3,4,5
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
372,0,0,0,0,0
373,0,0,0,0,0
374,0,0,0,0,0
375,0,0,0,0,0


In [106]:
model.run_test()

KeyboardInterrupt: 

In [31]:
assets = {}
targets={}
for i in range(1,6):
    assets[i] = pd.read_csv(f'long_short_training_features_targets/sector 1 - tech/Asset_{i}_training_data.csv')
    targets[i] = assets[i]['target']
    assets[i].drop(columns = ['target'])
dataset = pd.concat([assets[i] for i in range(1,6)], axis = 1)

In [44]:
window = 10
newSet = pd.DataFrame()
for i in range(window, len(dataset)):
    features = dataset[i-window:i+1].mean()
    features = pd.DataFrame(features).T
    if len(newSet)==0:
        newSet = features
    else:
        newSet = pd.concat([newSet, features])